In [1]:
import pandas as pd

df = pd.read_json('/home/leesh/codeit_project/codeit-project-docker/notebooks/origin_data/2023-06-05.json')

In [2]:
df.head()

,Asia/Seoul,id,event_key,session_id,device_id,user_id,value,user_properties,hackle_properties,event_properties
0,2023-06-06T00:01:35.519Z,68f56e0b-4a74-4444-8c99-d5b4e9f41b2b,$session_start,7918DF48-A239-45CE-AC1E-7E2E356454C4,7918DF48-A239-45CE-AC1E-7E2E356454C4,NaN,0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
1,2023-06-06T00:16:11.790Z,e4572df7-3b39-445b-9653-19ff676cb66c,$session_start,02F51978-FDD9-4C66-AE8D-820943E00AEF,02F51978-FDD9-4C66-AE8D-820943E00AEF,NaN,0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
2,2023-06-06T00:27:07.180Z,fc00b675-848a-44d9-9a3c-81108214b774,$session_start,Ut9JqG0u4RQGihxUY3vjCTGgMIx2,E75FFCB7-5AAB-4674-AE34-7A231136C367,NaN,0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
3,2023-06-06T00:28:38.522Z,b95591f9-9291-40b5-91e6-83491f16566d,$session_start,92D752F2-E7B9-4C01-892B-C9B3F1416B74,92D752F2-E7B9-4C01-892B-C9B3F1416B74,NaN,0,{},"{'language': 'ko-KR', 'isapp': 'true', 'osvers...",{}
4,2023-06-06T00:44:06.441Z,d64634ec-34eb-4797-b7f8-db16ad1bab76,$session_start,47955d3d-b77f-47a6-9d3e-3fcb45a350df,47955d3d-b77f-47a6-9d3e-3fcb45a350df,NaN,0,{},"{'language': 'ko', 'isapp': 'true', 'osversion...",{}


In [3]:
# 주요 정보 추출
df_simple = df.copy()

# 딕셔너리 → Series로 분해 (중요한 컬럼만)
df_simple['language'] = df_simple['hackle_properties'].apply(lambda x: x.get('language'))
df_simple['devicemodel'] = df_simple['hackle_properties'].apply(lambda x: x.get('devicemodel'))


# 1. UTC 기준으로 파싱
df_simple['event_datetime'] = pd.to_datetime(df_simple['Asia/Seoul'], utc=True)

# 2. 한국시간(Asia/Seoul)으로 타임존 변환
df_simple['event_datetime'] = df_simple['event_datetime'].dt.tz_convert('Asia/Seoul')

# 최종 확인용 컬럼만 선택
df_simple = df_simple[['event_datetime','device_id', 'event_key', 'devicemodel', 'language']]

In [4]:
df_simple['event_datetime'] = df_simple['event_datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
df_simple.head()

,event_datetime,device_id,event_key,devicemodel,language
0,2023-06-06 09:01:35,7918DF48-A239-45CE-AC1E-7E2E356454C4,$session_start,"iPhone13,1",ko-KR
1,2023-06-06 09:16:11,02F51978-FDD9-4C66-AE8D-820943E00AEF,$session_start,"iPad13,8",ko-KR
2,2023-06-06 09:27:07,E75FFCB7-5AAB-4674-AE34-7A231136C367,$session_start,"iPad13,4",ko-KR
3,2023-06-06 09:28:38,92D752F2-E7B9-4C01-892B-C9B3F1416B74,$session_start,"iPhone15,2",ko-KR
4,2023-06-06 09:44:06,47955d3d-b77f-47a6-9d3e-3fcb45a350df,$session_start,SM-A235N,ko


In [5]:
df_simple.to_parquet("/home/leesh/codeit_project/codeit-project-docker/parquet/230605.parquet", engine="pyarrow", compression="snappy")